## Imports

In [1]:
# Pandas is an open source data analysis and manipulation tool
import pandas as pd
from pandas import json_normalize

# os gives access to the operating system
import os
# The datetime module supplies classes for manipulating dates and times.
from datetime import datetime
import datetime
2
# This module provides various time-related functions.
import time

# Natural language toolkit
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

# library to create visualizations
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.backends.backend_pdf import PdfPages
from matplotlib.offsetbox import AnchoredText


# open source library for automating downloading of reports from Google Trends
from pytrends.request import TrendReq

# library to get html of website (wikipedia)
import requests
# json to use wikipedia return
import json

In [2]:
# Load exploded dataframe
path = "C:/Users/Jan/Documents/Python_Projects/Bachelorthesis/Bachelorthesis/Analysis/DataFrames/keyWords.csv"

keyWord_list = pd.read_csv(path, index_col=None,header=0)

keyWord_list.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 89441 entries, 0 to 89440
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   KeyWord    89441 non-null  object
 1   Occurence  89441 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 1.4+ MB


## Create list of keyWords with occurence above N

In [5]:
keyWord_list_n50 = keyWord_list[keyWord_list["Occurence"] >= 50]
keyWord_list_n50 = keyWord_list_n50.KeyWord.tolist()
#keyWord_list_n50 = keyWord_list_n50.reverse()
Google_DataFrame = get_google_interest_over_time(keyWord_list_n50)
Google_DataFrame

18-Jähriger
19-Jähriger


ResponseError: The request failed: Google returned a response with code 429.

In [ ]:
keyWord_list_n25.loc[keyWord_list_n25["KeyWord"] == "Kinder"]

In [3]:
error_keyWords_google_trends = []
successfull_keyWords = []
successfull_data = []


def get_google_interest_over_time(keyword_list):
    # connect to google
    pytrends = TrendReq(hl='de-DE', tz='60',timeout=120)

    # keywords
    keywords = keyword_list

    # build payload
    startDate = "2022-01-10"
    dateTime=datetime.datetime.now()
    currentDate = dateTime.strftime("%Y-%m-%d")
    timeframe = startDate +" "+ str(currentDate)

    pages_found_counter = 0
    pages_not_found_counter = 0

    sleep_counter = 0
    for keyword in keywords:
        print(keyword)
        pytrends.build_payload([keyword], cat=0, timeframe= str(timeframe) )

        # get data with interest over time
        try:
            data = pytrends.interest_over_time()
            successfull_data.append(data)
            successfull_keyWords.append(keyword)
            pages_found_counter = pages_found_counter + 1


        except:
            print(keyword)
            pages_not_found_counter = pages_not_found_counter + 1
        # sleeper to avoid bot detection
        time.sleep(5)
        sleep_counter = sleep_counter +1
        if sleep_counter == 100:
            time.sleep(30)
            sleep_counter == 0

    # concat all google trends results
    dfs = pd.concat(successfull_data, axis=1)
    # remove deprecated column
    dfs=dfs.loc[:,~dfs.columns.str.startswith('isPartial')]

    # get df to right format
    google_df = dfs.unstack().reset_index()
    google_df = google_df.rename(columns={"level_0" : "KeyWord", 0 : "Occurence_in_Google"})

    #fill all NaN with 0
    google_df["Occurence_in_Google"] = google_df["Occurence_in_Google"].fillna(0)
    google_df["Occurence_in_Google"] = google_df["Occurence_in_Google"].astype(int)

    google_df["normalized_Occurence_in_Google"] = google_df["Occurence_in_Google"].div(100)
    print(f"Successfull: {pages_found_counter}")
    print(f"Unsuccessfull: {pages_not_found_counter}")
    return google_df